In [6]:
import numpy as np
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.metrics import classification_report
from collections import Counter
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, fbeta_score

In [7]:
train_data_loaded = pd.read_csv('../../data/train_data_2024-08-01.csv')

X = train_data_loaded.drop(columns=['UKATEGORIE'])
y = train_data_loaded['UKATEGORIE']

In [8]:
# StratifiedKFold initialisieren
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# Variablen für Ergebnisse
fbetas_rf = []
fbetas_random = []
fbetas_mehrheit = []

In [9]:
beta = 2
fbeta_scorer = make_scorer(fbeta_score, beta=beta)

In [ ]:
# Balanced Random Forest Modell mit aktualisierten Parametern
bal_rf_model =  BalancedRandomForestClassifier(n_estimators=2060, max_depth=16, max_features='sqrt', min_samples_leaf=2, min_samples_split=10, random_state=42, replacement=True, bootstrap=False,  class_weight={0:1, 1: 2})

rf_scores = cross_val_score(bal_rf_model, X, y, cv=skf, scoring=fbeta_scorer)
print(f'Durchschnittlicher Random Forest F-beta Score (mean) mit unseren Daten: {rf_scores.mean()}')
print(f'Standardabweichung des Random Forest F-beta Score (std) mit unseren Daten: {rf_scores.std()}')

# zufällige und Mehrheitsklassen-Vorhersagen
counter = 1
for train_index, test_index in skf.split(X, y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
    # Drucke die ersten 10 values von y_train
    print("nur split y_train:", y_train.values[:100])
    # Drucke die ersten 10 values von y_test
    print("nur split y_test:", y_test.values[:100])
    
    # Random Forest Modell trainieren
    bal_rf_model.fit(X_train, y_train)
    y_pred_rf = bal_rf_model.predict(X_test)
    print("Ersten 10 Vorhersagen mit dem Random Forest Modell:", y_pred_rf[:10])
    
    # Zufällige Vorhersagen basierend auf den Klassenwahrscheinlichkeiten erstellen
    class_counts = Counter(y_test)
    total_samples = len(y_test)
    class_probabilities = {cls: count / total_samples for cls, count in class_counts.items()}
    
    np.random.seed(42)
    y_pred_random_weighted = np.random.choice(
        list(class_probabilities.keys()),
        size=y_test.shape,
        p=list(class_probabilities.values())
    )
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem _random_weighted:", y_pred_random_weighted[:10])
    
    # Leistung der gewichteten zufälligen Vorhersagen bewerten
    fbeta_random_weighted = fbeta_score(y_test, y_pred_random_weighted, beta=beta)
    print(f"Fold Nummer {counter}: fbeta_random_weighted - Score ist {fbeta_random_weighted}")
    fbetas_random.append(fbeta_random_weighted)
    
    # Immer Mehrheitsklasse vorhersagen
    y_pred_majority = np.full(y_test.shape, y_train.mode().iloc[0])
    
    # Drucke die ersten 10 Werte
    print("Ersten 10 Vorhersagen mit dem y_pred_majority:", y_pred_majority[:10])
    
    # Leistung der Vorhersagen der Mehrheitsklasse bewerten
    fbeta_mehrheit = fbeta_score(y_test, y_pred_majority, beta=beta)
    print(f"Fold Nummer {counter}: fbeta_mehrheit - Score ist {fbeta_mehrheit}")
    fbetas_mehrheit.append(fbeta_mehrheit)
    
    counter += 1

# Durchschnittliche F-beta-Scores berechnen
print(f'Random Weighted Predictions F-beta Score (mean): {np.mean(fbetas_random)}')
print(f'Random Weighted Predictions F-beta Score (std): {np.std(fbetas_random)}')

print(f'Majority Class Predictions F-beta Score (mean): {np.mean(fbetas_mehrheit)}')
print(f'Majority Class Predictions F-beta Score (std): {np.std(fbetas_mehrheit)}')

    

/home/kasch/Nextcloud/IKT/venv/lib/python3.12/site-packages/imblearn/ensemble/_forest.py:577: FutureWarning: The default of `sampling_strategy` will change from `'auto'` to `'all'` in version 0.13. This change will follow the implementation proposed in the original paper. Set to `'all'` to silence this warning and adopt the future behaviour.
  warn(


In [11]:
# Durchschnittliche Genauigkeiten berechnen
mean_scores_rf = np.mean(rf_scores)
mean_scores_random = np.mean(fbetas_random)
mean_scores_mehrheit = np.mean(fbetas_mehrheit)

In [12]:
print(f"Mean score: {mean_scores_rf}")
print(f"Weighted Random Prediction Mean score: {mean_scores_random}")
print(f"Weighted Mehrheit Prediction Mean score: {mean_scores_mehrheit}")


Mean score: 0.5012302914183395
Weighted Random Prediction Mean score: 0.7426462385123551
Weighted Mehrheit Prediction Mean score: 0.8167984428179098


In [10]:

# Klassifikationsberichte für den letzten Fold ausgeben
print("Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_rf))

print("Weighted Random Prediction Classification Report (Last Fold):")
print(classification_report(y_test, y_pred_random_weighted))

Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.93      0.31      0.47      6787
           1       0.19      0.86      0.30      1231

    accuracy                           0.40      8018
   macro avg       0.56      0.59      0.39      8018
weighted avg       0.81      0.40      0.44      8018

Weighted Random Prediction Classification Report (Last Fold):
              precision    recall  f1-score   support

           0       0.85      0.85      0.85      6787
           1       0.16      0.15      0.15      1231

    accuracy                           0.74      8018
   macro avg       0.50      0.50      0.50      8018
weighted avg       0.74      0.74      0.74      8018

